In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import os

/home/arnaud/workspace/Lab/IAVerte/green-ai/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/arnaud/workspace/Lab/IAVerte/green-ai/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/arnaud/workspace/Lab/IAVerte/green-ai/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/arnaud/worksp

In [2]:
print(tf.__version__)

1.4.0


## Load

In [3]:
from __future__ import division, print_function, absolute_import

import numpy as np
import tensorflow as tf
import time

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Training Parameters
num_gpus = 2
num_steps = 4000
learning_rate = 0.001
batch_size = 1024
display_step = 10

# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units


# Build a convolutional neural network
def conv_net(x, n_classes, dropout, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        # Convolution Layer with 64 filters and a kernel size of 5
        x = tf.layers.conv2d(x, 64, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        x = tf.layers.max_pooling2d(x, 2, 2)

        # Convolution Layer with 256 filters and a kernel size of 5
        x = tf.layers.conv2d(x, 256, 3, activation=tf.nn.relu)
        # Convolution Layer with 512 filters and a kernel size of 5
        x = tf.layers.conv2d(x, 512, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        x = tf.layers.max_pooling2d(x, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        x = tf.contrib.layers.flatten(x)

        # Fully connected layer (in contrib folder for now)
        x = tf.layers.dense(x, 2048)
        # Apply Dropout (if is_training is False, dropout is not applied)
        x = tf.layers.dropout(x, rate=dropout, training=is_training)

        # Fully connected layer (in contrib folder for now)
        x = tf.layers.dense(x, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        x = tf.layers.dropout(x, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(x, n_classes)
        # Because 'softmax_cross_entropy_with_logits' loss already apply
        # softmax, we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out

    return out


def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads


# By default, all variables will be placed on '/gpu:0'
# So we need a custom device function, to assign all variables to '/cpu:0'
# Note: If GPUs are peered, '/gpu:0' can be a faster option
PS_OPS = ['Variable', 'VariableV2', 'AutoReloadVariable']

def assign_to_device(device, ps_device='/cpu:0'):
    def _assign(op):
        node_def = op if isinstance(op, tf.NodeDef) else op.node_def
        if node_def.op in PS_OPS:
            return "/" + ps_device
        else:
            return device

    return _assign

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
num_gpus

2

In [5]:
# Place all ops on CPU by default
start_time = time.time()
with tf.device('/cpu:0'):
    tower_grads = []
    reuse_vars = False

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, num_input])
    Y = tf.placeholder(tf.float32, [None, num_classes])

    # Loop over all GPUs and construct their own computation graph
    for i in range(num_gpus):
        with tf.device(assign_to_device('/gpu:{}'.format(i), ps_device='/cpu:0')):

            # Split data between GPUs
            _x = X[i * batch_size: (i+1) * batch_size]
            _y = Y[i * batch_size: (i+1) * batch_size]

            # Because Dropout have different behavior at training and prediction time, we
            # need to create 2 distinct computation graphs that share the same weights.

            # Create a graph for training
            logits_train = conv_net(_x, num_classes, dropout,
                                    reuse=reuse_vars, is_training=True)
            # Create another graph for testing that reuse the same weights
            logits_test = conv_net(_x, num_classes, dropout,
                                   reuse=True, is_training=False)

            # Define loss and optimizer (with train logits, for dropout to take effect)
            loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=logits_train, labels=_y))
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            grads = optimizer.compute_gradients(loss_op)

            # Only first GPU compute accuracy
            if i == 0:
                # Evaluate model (with test logits, for dropout to be disabled)
                correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(_y, 1))
                accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

            reuse_vars = True
            tower_grads.append(grads)

    tower_grads = average_gradients(tower_grads)
    train_op = optimizer.apply_gradients(tower_grads)

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

    # Start Training
    with tf.Session() as sess:

        # Run the initializer
        sess.run(init)

        # Keep training until reach max iterations
        for step in range(1, num_steps + 1):
            # Get a batch for each GPU
            batch_x, batch_y = mnist.train.next_batch(batch_size * num_gpus)
            # Run optimization op (backprop)
            ts = time.time()
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            te = time.time() - ts
            if step % display_step == 0 or step == 1:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                     Y: batch_y})
                print("Step " + str(step) + ": Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc) + ", %i Examples/sec" % int(len(batch_x)/te))
            step += 1
        print("Optimization Finished!")

        # Calculate accuracy for MNIST test images
        print("Testing Accuracy:", \
            np.mean([sess.run(accuracy, feed_dict={X: mnist.test.images[i:i+batch_size],
            Y: mnist.test.labels[i:i+batch_size]}) for i in range(0, len(mnist.test.images), batch_size)]))
print("Total time of training : %s" % (time.time() - start_time))

Step 1: Minibatch Loss= 2.0531, Training Accuracy= 0.339, 568 Examples/sec
Step 10: Minibatch Loss= 0.4414, Training Accuracy= 0.899, 15771 Examples/sec
Step 20: Minibatch Loss= 0.1711, Training Accuracy= 0.952, 19907 Examples/sec
Step 30: Minibatch Loss= 0.1483, Training Accuracy= 0.972, 21308 Examples/sec
Step 40: Minibatch Loss= 0.0820, Training Accuracy= 0.976, 21920 Examples/sec
Step 50: Minibatch Loss= 0.0824, Training Accuracy= 0.978, 22267 Examples/sec
Step 60: Minibatch Loss= 0.0523, Training Accuracy= 0.983, 22205 Examples/sec
Step 70: Minibatch Loss= 0.0448, Training Accuracy= 0.989, 20962 Examples/sec
Step 80: Minibatch Loss= 0.0562, Training Accuracy= 0.979, 20939 Examples/sec
Step 90: Minibatch Loss= 0.0478, Training Accuracy= 0.991, 20979 Examples/sec
Step 100: Minibatch Loss= 0.0365, Training Accuracy= 0.992, 21263 Examples/sec
Step 110: Minibatch Loss= 0.0313, Training Accuracy= 0.991, 21375 Examples/sec
Step 120: Minibatch Loss= 0.0176, Training Accuracy= 0.991, 20916

Step 1040: Minibatch Loss= 0.0015, Training Accuracy= 1.000, 20940 Examples/sec
Step 1050: Minibatch Loss= 0.0009, Training Accuracy= 1.000, 21019 Examples/sec
Step 1060: Minibatch Loss= 0.0054, Training Accuracy= 1.000, 20974 Examples/sec
Step 1070: Minibatch Loss= 0.0028, Training Accuracy= 1.000, 22095 Examples/sec
Step 1080: Minibatch Loss= 0.0009, Training Accuracy= 1.000, 22091 Examples/sec
Step 1090: Minibatch Loss= 0.0018, Training Accuracy= 1.000, 20906 Examples/sec
Step 1100: Minibatch Loss= 0.0088, Training Accuracy= 0.999, 20840 Examples/sec
Step 1110: Minibatch Loss= 0.0020, Training Accuracy= 1.000, 20852 Examples/sec
Step 1120: Minibatch Loss= 0.0008, Training Accuracy= 1.000, 21521 Examples/sec
Step 1130: Minibatch Loss= 0.0004, Training Accuracy= 1.000, 20938 Examples/sec
Step 1140: Minibatch Loss= 0.0005, Training Accuracy= 1.000, 22070 Examples/sec
Step 1150: Minibatch Loss= 0.0005, Training Accuracy= 1.000, 20839 Examples/sec
Step 1160: Minibatch Loss= 0.0030, Train

Step 2070: Minibatch Loss= 0.0058, Training Accuracy= 1.000, 21126 Examples/sec
Step 2080: Minibatch Loss= 0.0001, Training Accuracy= 1.000, 20854 Examples/sec
Step 2090: Minibatch Loss= 0.0004, Training Accuracy= 1.000, 22003 Examples/sec
Step 2100: Minibatch Loss= 0.0024, Training Accuracy= 1.000, 22088 Examples/sec
Step 2110: Minibatch Loss= 0.0013, Training Accuracy= 1.000, 21265 Examples/sec
Step 2120: Minibatch Loss= 0.0005, Training Accuracy= 0.999, 21825 Examples/sec
Step 2130: Minibatch Loss= 0.0023, Training Accuracy= 1.000, 21015 Examples/sec
Step 2140: Minibatch Loss= 0.0002, Training Accuracy= 1.000, 21219 Examples/sec
Step 2150: Minibatch Loss= 0.0004, Training Accuracy= 1.000, 21809 Examples/sec
Step 2160: Minibatch Loss= 0.0127, Training Accuracy= 1.000, 21041 Examples/sec
Step 2170: Minibatch Loss= 0.0015, Training Accuracy= 1.000, 21507 Examples/sec
Step 2180: Minibatch Loss= 0.0000, Training Accuracy= 1.000, 21007 Examples/sec
Step 2190: Minibatch Loss= 0.0012, Train

Step 3100: Minibatch Loss= 0.0008, Training Accuracy= 1.000, 21118 Examples/sec
Step 3110: Minibatch Loss= 0.0035, Training Accuracy= 1.000, 21338 Examples/sec
Step 3120: Minibatch Loss= 0.0001, Training Accuracy= 1.000, 21900 Examples/sec
Step 3130: Minibatch Loss= 0.0113, Training Accuracy= 1.000, 22278 Examples/sec
Step 3140: Minibatch Loss= 0.0110, Training Accuracy= 1.000, 21135 Examples/sec
Step 3150: Minibatch Loss= 0.0000, Training Accuracy= 1.000, 21133 Examples/sec
Step 3160: Minibatch Loss= 0.0000, Training Accuracy= 1.000, 21250 Examples/sec
Step 3170: Minibatch Loss= 0.0010, Training Accuracy= 1.000, 21151 Examples/sec
Step 3180: Minibatch Loss= 0.0021, Training Accuracy= 0.999, 22159 Examples/sec
Step 3190: Minibatch Loss= 0.0017, Training Accuracy= 1.000, 21102 Examples/sec
Step 3200: Minibatch Loss= 0.0006, Training Accuracy= 1.000, 21939 Examples/sec
Step 3210: Minibatch Loss= 0.0002, Training Accuracy= 1.000, 20881 Examples/sec
Step 3220: Minibatch Loss= 0.0003, Train

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 446065761015725079
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 162004992
locality {
  bus_id: 1
}
incarnation: 7238694998269149715
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 161611776
locality {
  bus_id: 1
}
incarnation: 7633307642351845955
physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:41:00.0, compute capability: 6.1"
]
